In [22]:
import gym
import random
import copy
import numpy as np
from collections import deque
from gym.spaces.box import Box
from gym import wrappers
from gym.wrappers import TransformObservation
import itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader, IterableDataset

from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

from gym.wrappers import RecordVideo, RecordEpisodeStatistics, TimeLimit, AtariPreprocessing


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
num_gpus = torch.cuda.device_count()

In [23]:
class DRQN(nn.Module):
    def __init__(self, state_size , n_actions):
        super(DRQN, self).__init__()
        
        self.state_size = state_size
        self.seq = nn.Sequential(
                     nn.Linear(state_size[0], 128),
                     nn.ReLU(),
                     nn.Linear(128, 128),
                     nn.ReLU(),
                     nn.Linear(128, 128)    
                    )
        self.lstm = nn.LSTMCell(128, 256)
        self.linear = nn.Linear(256, n_actions)
        
   
    def forward(self, x, prev_state=None):        
        o = self.seq(x.float())
        if prev_state is not None:
            hs, ct = self.lstm(o, prev_state)
        else:
            hs = torch.zeros(x.shape[0], 256).to(device)
            ct = torch.zeros(x.shape[0], 256).to(device)
            hs, ct = self.lstm(o, (hs,ct))    
        q_values = self.linear(hs)
        return q_values, (hs, ct)

In [24]:
def epsilon_greedy(state, env, net, hidden=None, epsilon=0.0):
    if np.random.random() < epsilon:
        action = env.action_space.sample()
    else:
        state = state.to(device)
        q_values, hidden = net(state, hidden)
        _, action = torch.max(q_values, dim=1)
        action = int(action.item())
    return action, hidden

In [25]:
def slice_deque(buffer, start, stop, step):
    buffer.rotate(-start)
    slice = list(itertools.islice(buffer, 0, stop-start, step))
    buffer.rotate(start)
    return slice

class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
        
    def __len__(self):
        return len(self.buffer)
    
    def append(self, experience):
        self.buffer.append(experience)
        
    def sample(self, sample_size, sequence_len):
        batch = []
        while len(batch) < sample_size :
            start = random.randint(0, len(self.buffer) - sequence_len)
            sequenze_sample = slice_deque(self.buffer, start, start + sequence_len, 1)
            batch += sequenze_sample
            
        return batch

In [26]:
class RLDataset(IterableDataset):
    def __init__(self, buffer, sample_size=400, sequence_len=5):
        self.buffer = buffer
        self.sample_size = sample_size
        self.sequence_len = sequence_len
        
    def __iter__(self):
        for experience in self.buffer.sample(self.sample_size,self.sequence_len):
            yield experience

In [27]:
def create_environment(name):
    env = gym.make(name, render_mode="rgb_array")
    env = TimeLimit(env, max_episode_steps=400)
    env = RecordVideo(env, video_folder='./videos/DRQN_LunarLander', episode_trigger=lambda x: x % 100 == 0)
    env = RecordEpisodeStatistics(env)
    return env



In [28]:
class DeepQLearning(LightningModule):
    def __init__(self, env_name, policy=epsilon_greedy, capacity=100_000, 
               batch_size=256, lr=1e-3, hidden_size=128, gamma=0.99, 
               loss_fn=F.smooth_l1_loss, optim=AdamW, eps_start=1.0, eps_end=0.15, 
               eps_last_episode=100, samples_per_epoch=1024, sync_rate=10,
               sequence_length = 4):
    
        super().__init__()
        self.env = create_environment(env_name)

        obs_size = self.env.observation_space.shape
        n_actions = self.env.action_space.n

        self.q_net = DRQN(obs_size, n_actions)

        self.target_q_net = copy.deepcopy(self.q_net)

        self.policy = policy
        self.buffer = ReplayBuffer(capacity=capacity)
        self.save_hyperparameters()

        while len(self.buffer) < self.hparams.samples_per_epoch:
            print(f"{len(self.buffer)} samples in experience buffer. Filling...")
            self.play_episode(epsilon=self.hparams.eps_start)
            
    @torch.no_grad()
    def play_episode(self, policy=None, epsilon=0.):
        state  = self.env.reset()
        state  = torch.from_numpy(state[0])
        hidden = None
        done = False
        step = 0
        while not done:
            if policy:
                action, hidden = policy(state.unsqueeze(dim=0), self.env, self.q_net, hidden, epsilon=epsilon)
            else:
                action = self.env.action_space.sample()
            next_state, reward, done, tru , _ = self.env.step(action)
            if tru:
                done = tru
            step += 1 
            next_state = torch.from_numpy(next_state)   
            exp = (step, state, action, reward, done, next_state)
            self.buffer.append(exp)
            state = next_state
            
        self.env.close()
        
        
    def forward(self, x):
        return self.q_net(x)

    
    def configure_optimizers(self):
        q_net_optimizer = self.hparams.optim(self.q_net.parameters(), lr=self.hparams.lr)
        return [q_net_optimizer]

     # Create dataloader.
    def train_dataloader(self):
        dataset = RLDataset(self.buffer, self.hparams.samples_per_epoch, self.hparams.sequence_length)
       
        dataloader = DataLoader(
            dataset=dataset,
            batch_size=self.hparams.batch_size * self.hparams.sequence_length
        )
        return dataloader
    
    def training_step(self, batch, batch_idx):
        steps, states, actions, rewards, dones, next_states = batch
        actions = actions.unsqueeze(1)
        rewards = rewards.unsqueeze(1)
        dones = dones.unsqueeze(1)
        
        stack_q_values = []
        stack_actions = []
        stack_rewards = []
        stack_dones = []
        hidden = None
        for i in range(self.hparams.sequence_length):
            stack_actions.append(actions[i::self.hparams.sequence_length])
            stack_dones.append(dones[i::self.hparams.sequence_length])
            stack_rewards.append(rewards[i::self.hparams.sequence_length])
            q_values, hidden  = self.q_net(states[i::self.hparams.sequence_length],hidden)
            stack_q_values.append(q_values)
        stack_q_values = torch.cat(stack_q_values,dim=0)
        stack_actions = torch.cat(stack_actions,dim=0)
        stack_dones = torch.cat(stack_dones,dim=0)
        stack_rewards = torch.cat(stack_rewards,dim=0)

    
        state_action_values = torch.gather(stack_q_values, -1, stack_actions)
        
        
        stack_next_q_values = []
        hidden = None
        for i in range(self.hparams.sequence_length):
            q_values, hidden = self.target_q_net(next_states[i::self.hparams.sequence_length],hidden)
            stack_next_q_values.append(q_values)
                
        stack_next_q_values = torch.cat(stack_next_q_values,dim=0)
        next_action_values = torch.max(stack_next_q_values, dim=1)[0].unsqueeze(dim=1)
        next_action_values[stack_dones] = 0.0

        expected_state_action_values = stack_rewards + self.hparams.gamma * next_action_values

        loss = self.hparams.loss_fn(state_action_values, expected_state_action_values)
        self.log('episode/Q-Error', loss)
        return loss
    
    # Training epoch end.
    def training_epoch_end(self, training_step_outputs):
        epsilon = max(
            self.hparams.eps_end,
            self.hparams.eps_start - self.current_epoch / self.hparams.eps_last_episode
        )

        self.play_episode(policy=self.policy, epsilon=epsilon)
        self.log('episode/Return', self.env.return_queue[-1])

        if self.current_epoch % self.hparams.sync_rate == 0:
            self.target_q_net.load_state_dict(self.q_net.state_dict())
            
    def save_model(self):
        torch.save(self.q_net.state_dict(), "./model")
        
    def load_model(self):
        self.q_net.load_state_dict(torch.load( "./model"))


In [29]:
algo = DeepQLearning('LunarLander-v2')

checkpoint_callback = ModelCheckpoint(dirpath="./checkpoints/DRQN-LunarLander", save_top_k=1,mode="max", monitor="episode/Return")

trainer = Trainer(
     accelerator='gpu',
     devices=num_gpus,
     max_epochs=10_000,
     callbacks=[checkpoint_callback,EarlyStopping(monitor='episode/Return', mode='max', patience=500)]
)

trainer.fit(algo)

D:\multi_agents_rl\env\lib\site-packages\gym\wrappers\record_video.py:75: UserWarning: WARN: Overwriting existing videos at D:\multi_agents_rl\videos\DRQN_LunarLander folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


0 samples in experience buffer. Filling...
Moviepy - Building video D:\multi_agents_rl\videos\DRQN_LunarLander\rl-video-episode-0.mp4.
Moviepy - Writing video D:\multi_agents_rl\videos\DRQN_LunarLander\rl-video-episode-0.mp4



GPU available: True (cuda), used: True                                                                                                                                         
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
D:\multi_agents_rl\env\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:616: UserWarning: Checkpoint directory D:\multi_agents_rl\checkpoints\DRQN-LunarLander exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type | Params
--------------------------------------
0 | q_net        | DRQN | 430 K 
1 | target_q_net | DRQN | 430 K 
--------------------------------------
860 K     Trainable params
0         Non-trainable params
860 K     Total params
3.444     Total estimated model params size (MB)


Moviepy - Done !
Moviepy - video ready D:\multi_agents_rl\videos\DRQN_LunarLander\rl-video-episode-0.mp4
111 samples in experience buffer. Filling...
200 samples in experience buffer. Filling...
312 samples in experience buffer. Filling...
429 samples in experience buffer. Filling...
525 samples in experience buffer. Filling...
611 samples in experience buffer. Filling...
750 samples in experience buffer. Filling...
874 samples in experience buffer. Filling...
945 samples in experience buffer. Filling...
1015 samples in experience buffer. Filling...


D:\multi_agents_rl\env\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Moviepy - Done !                                          
Moviepy - video ready D:\multi_agents_rl\videos\DRQN_LunarLander\rl-video-episode-1300.mp4
Moviepy - Building video D:\multi_agents_rl\videos\DRQN_LunarLander\rl-video-episode-1400.mp4.
Moviepy - Writing video D:\multi_agents_rl\videos\DRQN_LunarLander\rl-video-episode-1400.mp4

Epoch 1388: : 1it [00:00,  1.49it/s, loss=0.985, v_num=12]


t:   0%|                                                                                                                                       | 0/1 [00:00<?, ?it/s, now=None]
                                                                                                                                                                               

Moviepy - Done !                                          
Moviepy - video ready D:\multi_agents_rl\videos\DRQN_LunarLander\rl-video-episode-1400.mp4
Moviepy - Building video D:\multi_agents_rl\videos\DRQN_LunarLander\rl-video-episode-1400.mp4.
Moviepy - Writing video D:\multi_agents_rl\videos\DRQN_LunarLander\rl-video-episode-1400.mp4

Epoch 1389: : 1it [00:02,  2.34s/it, loss=1.01, v_num=12]


t:  89%|████████████████████████████████████████████████████████████████████████████████████████████████████████████▏             | 258/291 [00:00<00:00, 486.91it/s, now=None]
                                                                                                                                                                               

Moviepy - Done !                                         
Moviepy - video ready D:\multi_agents_rl\videos\DRQN_LunarLander\rl-video-episode-1400.mp4
Epoch 1482: : 1it [00:00,  1.80it/s, loss=1.01, v_num=12] 


In [21]:
policy = algo.policy
algo.q_net = algo.q_net.to(device)

for i in range(100):
    print(i)
    algo.play_episode(policy=algo.policy, epsilon=0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
Moviepy - Building video D:\multi_agents_rl\videos\DRQN_LunarLander\rl-video-episode-2000.mp4.
Moviepy - Writing video D:\multi_agents_rl\videos\DRQN_LunarLander\rl-video-episode-2000.mp4



Moviepy - Done !
Moviepy - video ready D:\multi_agents_rl\videos\DRQN_LunarLander\rl-video-episode-2000.mp4
18


Moviepy - Building video D:\multi_agents_rl\videos\DRQN_LunarLander\rl-video-episode-2000.mp4.
Moviepy - Writing video D:\multi_agents_rl\videos\DRQN_LunarLander\rl-video-episode-2000.mp4



Moviepy - Done !
Moviepy - video ready D:\multi_agents_rl\videos\DRQN_LunarLander\rl-video-episode-2000.mp4
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


KeyboardInterrupt: 